In [403]:
import numpy as np
import math


In [404]:
# def generateKeyMatrix(n, modulo):
#     while True:
#         key_matrix = np.random.randint(1, modulo, size=(n, n))
#         try:
#             inverse_key_matrix = inverse(key_matrix, modulo)
#             return key_matrix, inverse_key_matrix
#         except ValueError:
#             continue  # Regenerate if not invertible

In [405]:

def generateKeyMatrix(n, modulo):
    while True:
        key_matrix = np.random.randint(1, modulo, size=(n, n))
        det = int(round(np.linalg.det(key_matrix))) % modulo
        # Check if gcd(det, modulo) == 1 to ensure invertibility
        if math.gcd(det, modulo) == 1:
            try:
                inverse_key_matrix = inverse_2x2_mod(key_matrix, modulo)
                return key_matrix, inverse_key_matrix
            except ValueError:
                continue
        else:
            print("Failed Iteration")

In [406]:
def inverse(matrix, modulo):
    det = int(round(np.linalg.det(matrix))) % modulo
    det_inv = pow(det, -1, modulo)  # Modular inverse of determinant
    adjugate = np.round(np.linalg.inv(matrix) * det).astype(int) % modulo
    return (det_inv * adjugate) % modulo

In [407]:
def inverse_2x2_mod(matrix, modulo):
    a, b = matrix[0]
    c, d = matrix[1]
    det = int((a*d - b*c) % modulo)  # Ensure det is a Python int
    if math.gcd(det, modulo) != 1:
        raise ValueError("Matrix not invertible modulo {}".format(modulo))
    
    det_inv = pow(det, -1, modulo)  # Now det is a Python int, this should work
    # Adjugate matrix = [[d, -b], [-c, a]]
    return (det_inv * np.array([[d, -b],
                                [-c, a]])) % modulo


In [408]:
def encrypt(plaintext, key_matrix, modulo, alphabet):
    # Map characters to numbers
    char_to_num = {char: idx + 1 for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}

    # Convert plaintext to numbers and pair into groups
    numbers = [char_to_num[char] for char in plaintext if char in char_to_num]
    if len(numbers) % 2 != 0:
        numbers.append(0)  # Padding if necessary

    plaintext_matrix = np.array(numbers).reshape(-1, 2).T
    print("Original Matrix: \n", np.array(numbers).reshape(-1, 2))
    ciphertext_matrix = (np.dot(key_matrix, plaintext_matrix) % modulo).T

    # Convert numbers back to characters
    ciphertext = ''.join(num_to_char.get(num, '') for pair in ciphertext_matrix for num in pair if num != 0)
    return ciphertext, ciphertext_matrix

In [409]:
def decrypt(ciphertext, inverse_key_matrix, modulo, alphabet):
    # Map numbers back to plaintext
    char_to_num = {char: idx + 1 for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}

    numbers = [char_to_num[char] for char in ciphertext if char in char_to_num]
    if len(numbers) % 2 != 0:
        numbers.append(0)  # Padding if necessary

    cipher_matrix = np.array(numbers).reshape(-1, 2)

    decrypted_matrix = (np.dot(inverse_key_matrix, cipher_matrix.T) % modulo).T
    text = ''.join(num_to_char.get(int(round(num)), '') for pair in decrypted_matrix for num in pair if num != 0)
    return text

In [410]:
modulo = 58  # Adjust modulo to match extended alphabet size
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?"

plaintext = "Hello My name is Nic"
key_matrix = np.array([
    [3, 2],
    [5, 7]
])

# Compute inverse key matrix dynamically
inverse_key_matrix = inverse(key_matrix, modulo)
print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

# Encrypt the plaintext
ciphertext, ciphertext_matrix = encrypt(plaintext, key_matrix, modulo, alphabet)
print(f"Ciphertext Matrix:\n{ciphertext_matrix}")

# Decrypt the ciphertext
decrypted_text = decrypt(ciphertext, inverse_key_matrix, modulo, alphabet)
print("Original Text:", plaintext)
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted Text: {decrypted_text}")

ciphertext, ciphertext_matrix = encrypt(plaintext, inverse_key_matrix, modulo, alphabet)
decrypted_text = decrypt(ciphertext, key_matrix, modulo, alphabet)
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted Text: {decrypted_text}")

Key Matrix:
[[3 2]
 [5 7]]
Inverse Key Matrix:
[[27 42]
 [47 53]]
Original Matrix: 
 [[ 8 31]
 [38 38]
 [41 53]
 [13 51]
 [53 40]
 [27 39]
 [31 53]
 [35 45]
 [53 14]
 [35 29]]
Ciphertext Matrix:
[[28 25]
 [16 50]
 [55 54]
 [25 16]
 [ 7 23]
 [43  2]
 [25  4]
 [21 26]
 [13 15]
 [47 30]]
Original Text: Hello My name is Nic
Ciphertext: bYPx,.YPGWqBYDUZMOud
Decrypted Text: Hello My name is Nic
Original Matrix: 
 [[ 8 31]
 [38 38]
 [41 53]
 [13 51]
 [53 40]
 [27 39]
 [31 53]
 [35 45]
 [53 14]
 [35 29]]
Ciphertext: JuLdal?HkcudufybuqQx
Decrypted Text: Hello My name is Nic


In [411]:
modulo = 58  # Adjust modulo to match extended alphabet size
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?"

plaintext = "Hello My name is Nic"
key_matrix, inverse_key_matrix = generateKeyMatrix(2, modulo)

# Compute inverse key matrix dynamically

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

# Encrypt the plaintext
ciphertext, ciphertext_matrix = encrypt(plaintext, key_matrix, modulo, alphabet)
print(f"Ciphertext Matrix:\n{ciphertext_matrix}")

# Decrypt the ciphertext
decrypted_text = decrypt(ciphertext, inverse_key_matrix, modulo, alphabet)
print("Original Text:", plaintext)
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted Text: {decrypted_text}")

Key Matrix:
[[37 14]
 [14 23]]
Inverse Key Matrix:
[[15  6]
 [ 6  9]]
Original Matrix: 
 [[ 8 31]
 [38 38]
 [41 53]
 [13 51]
 [53 40]
 [27 39]
 [31 53]
 [35 45]
 [53 14]
 [35 29]]
Ciphertext Matrix:
[[34 13]
 [24 14]
 [55 53]
 [35 21]
 [27 38]
 [37 57]
 [33 29]
 [11 17]
 [11 20]
 [19 55]]
Original Text: Hello My name is Nic
Ciphertext: hMXN, iUalk?gcKQKTS,
Decrypted Text: Hello My name is Nic


In [412]:
def minor_mod(M, i, j, mod):
    # Return the minor matrix M_(i,j) with the ith row and jth column removed
    return np.delete(np.delete(M, i, axis=0), j, axis=1) % mod

In [413]:
def determinant_mod(M, mod):
    # Compute the determinant of M mod 'mod' using recursion (Laplace expansion)
    M = M % mod
    n = M.shape[0]
    if n == 1:
        return M[0,0] % mod
    elif n == 2:
        return (M[0,0]*M[1,1] - M[0,1]*M[1,0]) % mod

    det_val = 0
    for j in range(n):
        cofactor = ((-1)**j) * M[0,j] * determinant_mod(minor_mod(M, 0, j, mod), mod)
        det_val = (det_val + cofactor) % mod
    return det_val % mod

In [414]:
def adjugate_mod(M, mod):
    n = M.shape[0]
    adj = np.zeros_like(M, dtype=int)
    for i in range(n):
        for j in range(n):
            # Compute cofactor C[i,j]
            C_ij = ((-1)**(i+j)) * determinant_mod(minor_mod(M, i, j, mod), mod)
            # Note: adjugate is transpose of the cofactor matrix
            adj[j, i] = C_ij % mod
    return adj % mod

In [415]:
def inverse_nxn_mod(M, mod):
    # Compute the modular inverse of an n×n matrix M under modulo 'mod'
    M = M % mod
    det = determinant_mod(M, mod)
    if math.gcd(det, mod) != 1:
        raise ValueError("Matrix not invertible modulo {}".format(mod))
    det_inv = pow(int(det), -1, mod)
    adj = adjugate_mod(M, mod)
    return (det_inv * adj) % mod

In [416]:
def generateKeyMatrix(n, modulo):
    while True:
        key_matrix = np.random.randint(1, modulo, size=(n, n))
        det = determinant_mod(key_matrix, modulo)
        if math.gcd(det, modulo) == 1:
            try:
                inverse_key_matrix = inverse_nxn_mod(key_matrix, modulo)
                return key_matrix, inverse_key_matrix
            except ValueError:
                continue
        else:
            print("Failed Iteration")


In [417]:
def encryptNN(plaintext, key_matrix, modulo, alphabet):
    n = key_matrix.shape[0]
    char_to_num = {char: idx + 1 for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}

    numbers = [char_to_num[char] for char in plaintext if char in char_to_num]

    # Pad so length is a multiple of n
    while len(numbers) % n != 0:
        numbers.append(0)

    # Reshape into n-rows and as many columns as needed, then transpose
    plaintext_matrix = np.array(numbers).reshape(-1, n).T
    print("Plain Text Matrix: \n", plaintext_matrix)
    ciphertext_matrix = (np.dot(key_matrix, plaintext_matrix) % modulo).T

    # Convert numbers back to characters
    ciphertext = ''.join(num_to_char.get(num, '')
                         for row in ciphertext_matrix
                         for num in row if num != 0)
    return ciphertext, ciphertext_matrix

In [418]:


def decryptNN(ciphertext, inverse_key_matrix, modulo, alphabet):
    n = inverse_key_matrix.shape[0]
    char_to_num = {char: idx + 1 for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}

    numbers = [char_to_num[char] for char in ciphertext if char in char_to_num]

    # Pad if needed
    while len(numbers) % n != 0:
        numbers.append(0)

    cipher_matrix = np.array(numbers).reshape(-1, n)
    decrypted_matrix = (np.dot(inverse_key_matrix, cipher_matrix.T) % modulo).T

    text = ''.join(num_to_char.get(int(num), '')
                   for row in decrypted_matrix
                   for num in row if num != 0)
    return text


In [419]:
modulo = 58  # Adjust modulo to match extended alphabet size
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?"

plaintext = "Hello My name is Nic"
key_matrix, inverse_key_matrix = generateKeyMatrix(2, modulo)

# Compute inverse key matrix dynamically

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

# Encrypt the plaintext
ciphertext, ciphertext_matrix = encrypt(plaintext, key_matrix, modulo, alphabet)

# Decrypt the ciphertext
decrypted_text = decrypt(ciphertext, inverse_key_matrix, modulo, alphabet)
print("Original Text:", plaintext)
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted Text: {decrypted_text}")

Failed Iteration
Key Matrix:
[[47 55]
 [38 15]]
Inverse Key Matrix:
[[27 17]
 [36 15]]
Original Matrix: 
 [[ 8 31]
 [38 38]
 [41 53]
 [13 51]
 [53 40]
 [27 39]
 [31 53]
 [35 45]
 [53 14]
 [35 29]]
Original Text: Hello My name is Nic
Ciphertext: yOvpbgzoyDxsVABgMTxY
Decrypted Text: Hello My name is Nic


In [420]:
modulo = 58  # Adjust modulo to match extended alphabet size\
keyMatrixSize = 6
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?"

plaintext = "Hello My name is Nic"
key_matrix, inverse_key_matrix = generateKeyMatrix(keyMatrixSize, modulo)


# Compute inverse key matrix dynamically
print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

# Encrypt the plaintext
ciphertext, ciphertext_matrix = encryptNN(plaintext, key_matrix, modulo, alphabet)

# Decrypt the ciphertext
decrypted_text = decryptNN(ciphertext, inverse_key_matrix, modulo, alphabet)
print("Original Text:", plaintext)
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted Text: {decrypted_text}")

Failed Iteration
Failed Iteration
Key Matrix:
[[42 52 22 38 51  4]
 [23 49 32 51 19  6]
 [36 17 12 36 44 40]
 [30 12 26 47  6 39]
 [23  6 52 46  6 30]
 [13 34 47  8 37 45]]
Inverse Key Matrix:
[[ 2 12 44 54 17 26]
 [22  6 37 28 26 22]
 [16  7  5 49 12 51]
 [21 37 41 22 47 52]
 [35 28 20 42 34 22]
 [47 35  9 27 31 10]]
Plain Text Matrix: 
 [[ 8 13 31 35]
 [31 51 53 29]
 [38 53 35  0]
 [38 40 45  0]
 [41 27 53  0]
 [53 39 14  0]]
Original Text: Hello My name is Nic
Ciphertext: ilIONPyeKakrQruCgOTVMFyw
Decrypted Text: Hello My name is Nic
